In [ ]:
# | default_exp models

In [ ]:
# | export
from sqlmodel import Field, SQLModel, UniqueConstraint
from datetime import datetime
from seo_rat.sqlite_db import SQLiteDB
import re
from pydantic import field_validator
from pathlib import Path

In [ ]:
# | exporti
class Website(SQLModel, table=True):
    __table_args__ = {"extend_existing": True}
    id: int | None = Field(default=None, primary_key=True)
    url: str = Field(unique=True)
    name: str
    desc: str | None = None
    lang: str = "en"
    created_at: datetime = Field(default_factory=datetime.now)

    @field_validator("url")
    @classmethod
    def validate_url(cls, v: str) -> str:
        from urllip.parse import urlparse

        parsed = urlparse(v)
        if not all([parsed.scheme, parsed.netloc]):
            raise ValueError(f"Invalid URL: {v}")
        return v

    @field_validator("name")
    @classmethod
    def validate_name(cls, v: str) -> str:
        if not v or len(v.strip()) < 1:
            raise ValueError(f"Invalid Name: {v}")
        return v

    @field_validator("lang")
    @classmethod
    def validate_lang(cls, v: str) -> str:
        if not re.match(r"^[a-z]{2}(-[A-Z]{2})?$", v):
            raise ValueError(f"Invalid Language Code: {v}")
        return v

In [ ]:
# | hide
#| eval: false
db = SQLiteDB()
with db.get_session() as session:
    site = Website(
        url="https://kareemai.com",
        name="kareemai",
        lang="ar",
        desc="Kareem Personal Blog",
    )
    session.add(site)
    session.commit()
    print(f"Added: {site.name}")

Added: kareemai


In [ ]:
# | export
def get_all_websites(session):
    """Get all websites from database"""
    from sqlmodel import select

    return session.exec(select(Website)).all()


In [ ]:
# |hide
#| eval: false
get_all_websites(db.get_session())


[Website(id=2, desc='Kareem Personal Blog', created_at=datetime.datetime(2026, 1, 16, 2, 37, 49, 690924), lang='ar', url='https://kareemai.com', name='kareemai')]

In [ ]:
# | export
def delete_website(session, website_id: int):
    from sqlmodel import delete

    session.exec(delete(Website).where(Website.id == website_id))
    session.commit()


In [ ]:
# | hide
# delete_website(db.get_session(), website_id=1)

In [ ]:
# | exporti
class GSCAnalytics(SQLModel, table=True):
    __table_args__ = (
        UniqueConstraint("site_url", "date", "query", "page", "country", "device"),
        {"extend_existing": True},
    )
    id: int | None = Field(default=None, primary_key=True)
    site_url: str
    date: str
    query: str | None = None
    page: str | None = None
    country: str | None = None
    device: str | None = None
    clicks: int = 0
    impressions: int = 0
    ctr: float = 0.0
    position: float = 0.0
    created_at: datetime = Field(default_factory=datetime.now)


In [ ]:
# | exporti
class IndexStatus(SQLModel, table=True):
    __table_args__ = {"extend_existing": True}
    id: int | None = Field(default=None, primary_key=True)
    site_url: str
    page_url: str 
    verdict: str
    coverage_state: str | None = None
    last_crawl_time: str | None = None
    indexing_state: str | None = None
    robots_txt_state: str | None = None
    checked_at: datetime = Field(default_factory=datetime.now)
